In [4]:
print("Jupyter notebook is working!")

Jupyter notebook is working!


In [5]:
import pandas as pd
df = pd.read_excel('../data/online_retail_data.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
df = df.dropna(subset=['CustomerID'])

In [7]:
df["CustomerID"] = df["CustomerID"].astype(str)

In [8]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [9]:
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

In [10]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [11]:
import sqlite3

connection = sqlite3.connect('../data/online_retail.db')
df.to_sql('transactions', connection, if_exists='replace', index=False)
pd.read_sql_query("SELECT * FROM transactions LIMIT 5;", connection)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [12]:
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days = 1)
print("Snapshot date:", snapshot_date)

Snapshot date: 2011-12-10 12:50:00


In [13]:
query = f"""
SELECT 
    CustomerID,
    
    -- Number of unique orders
    COUNT(DISTINCT InvoiceNo) AS num_orders,
    
    -- Total money spent
    SUM(TotalPrice) AS total_spent,
    
    -- Average order value
    ROUND(SUM(TotalPrice) * 1.0 / COUNT(DISTINCT InvoiceNo), 2) AS avg_order_value,

    -- First and last purchase dates
    MIN(InvoiceDate) AS first_purchase_date,
    MAX(InvoiceDate) AS last_purchase_date,

    -- Tenure in days (from first to last order)
    JULIANDAY(MAX(InvoiceDate)) - JULIANDAY(MIN(InvoiceDate)) AS tenure_days,

    -- Recency in days (how long since last purchase at snapshot)
    JULIANDAY('{snapshot_date.date()}') - JULIANDAY(MAX(InvoiceDate)) AS recency_days,

    -- Average time between purchases
    CASE 
        WHEN COUNT(DISTINCT InvoiceNo) > 1 THEN 
            (JULIANDAY(MAX(InvoiceDate)) - JULIANDAY(MIN(InvoiceDate))) / (COUNT(DISTINCT InvoiceNo) - 1)
        ELSE NULL
    END AS avg_days_between_orders

FROM transactions
GROUP BY CustomerID;
"""


In [14]:

customer_summary = pd.read_sql_query(query, connection)
# Save the customer summary table to the database
customer_summary.to_sql('customer_features', connection, if_exists='replace', index=False)
customer_summary.head()

,CustomerID,num_orders,total_spent,avg_order_value,first_purchase_date,last_purchase_date,tenure_days,recency_days,avg_days_between_orders
0,12346.0,2,0.00,0.00,2011-01-18 10:01:00,2011-01-18 10:17:00,0.011111,325.571528,0.011111
1,12347.0,7,4310.00,615.71,2010-12-07 14:57:00,2011-12-07 15:52:00,365.038194,2.338889,60.839699
2,12348.0,4,1797.24,449.31,2010-12-16 19:09:00,2011-09-25 13:13:00,282.752778,75.449306,94.250926
3,12349.0,1,1757.55,1757.55,2011-11-21 09:51:00,2011-11-21 09:51:00,0.000000,18.589583,NaN
4,12350.0,1,334.40,334.40,2011-02-02 16:01:00,2011-02-02 16:01:00,0.000000,310.332639,NaN


In [15]:
customer_summary.to_sql('customer_features', connection, if_exists='replace', index=False)


4372

In [16]:
customer_summary.to_sql('customer_features', connection, if_exists='replace', index=False)


4372

In [18]:
import sqlite3
connection = sqlite3.connect('../data/online_retail.db')